In [ ]:
## This file implements neural networks and logistic regression on p0017SpresabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 84.82% for combination data, and 85.31% for over-sampling data 
## after improvements and regularization.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## However, with the new datasets, the accuracy becomes 87.39% for combination data and 80.42% for over-sampling data.
## For logistic regression, the accuracies are 96.43% for combination data and 94.41% for over-sampling data.
## For random forest, the accuracy is 98.21% for combination and 97.2% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.45% for combination data, and 97.58% for over-sampling.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017SpresabsSTCC_quant.csv')
df.shape

(255, 148)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0.214333
1      0.350500
2      0.517333
3      0.188833
4      0.191833
5      0.348667
6      0.182667
7      0.207000
8      0.405500
9      0.173500
10     0.184000
11     0.528667
12     0.211667
13     0.218667
14     0.167500
15     0.183333
16     0.406833
17     0.407333
18     0.321167
19     0.177167
20     0.271000
21     0.241833
22     0.178333
23     0.172500
24     0.205833
25     0.170000
26     0.224667
27     0.170500
28     0.163667
29     0.471000
         ...   
225    0.303700
226    0.310600
227    0.443300
228    0.388000
229    0.541200
230    0.378667
231    0.242167
232    0.261500
233    0.295833
234    0.193167
235    0.391333
236    0.160500
237    0.159333
238    0.172000
239    0.262000
240    0.244833
241    0.279833
242    0.200167
243    0.168333
244    0.548667
245    0.164167
246    0.202333
247    0.194167
248    0.188167
249    0.203333
250    0.444667
251    0.238000
252    0.167000
253    0.314333
254    0.252000
Name: pheno, Length: 255

In [5]:
df.head()

id  \
0     107   
1     109   
2     115   
3  120335   
4  120337   

   TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   TTTTCCAGTAAT  TTTTAATACATAT  TTTTAAATATTATAA  \
0             0              0                0   
1             1              1                1   
2             0              0                0   
3             0              0                0   
4             0              0                0   

   TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   TTTATCTTTATGA  TTTAATTTAGTAAGT  \
0              0                0   
1              0                1   
2              0                0   
3              0                0   
4              0                0   

   TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   ...  group_3904  group_426  group_475  group_6375  group_7822  group_8071  \
0  ...           0          0          0           0           0           0   
1  ...           0          0          0           0           0           0   
2  ...           0          0          0           0           0           0   
3  ...           0          0          0           0           0           0   
4  ...           0          0          0           0           0           0   

   group_8913     pheno  ST  CC  
0           0  0.214333   5   5  
1           0  0.350500   8   8  
2           0  0.517333   5   5  
3           0  0.188833   5   5  
4           0  0.191833   5   5  

[5 rows x 148 columns]

In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 148)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 147)

In [12]:
df_clean.head()

TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   TTTTCCAGTAAT  TTTTAATACATAT  TTTTAAATATTATAA  \
0             0              0                0   
1             1              1                1   
2             0              0                0   
3             0              0                0   
4             0              0                0   

   TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   TTTATCTTTATGA  TTTAATTTAGTAAGT  \
0              0                0   
1              0                1   
2              0                0   
3              0                0   
4              0                0   

   TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC  \
0                                                  1                                                      
1                                                  1                                                      
2                                                  1                                                      
3                                                  1                                                      
4                                                  1                                                      

   TTCCATCGAATCAC  ...  group_3904  group_426  group_475  group_6375  \
0               0  ...           0          0          0           0   
1               1  ...           0          0          0           0   
2               0  ...           0          0          0           0   
3               0  ...           0          0          0           0   
4               0  ...           0          0          0           0   

   group_7822  group_8071  group_8913  pheno  ST  CC  
0           0           0           0      0   5   5  
1           0           0           0      0   8   8  
2           0           0           0      1   5   5  
3           0           0           0      0   5   5  
4           0           0           0      0   5   5  

[5 rows x 147 columns]

In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 146) (255,)


In [14]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 191), (1, 180)]


Using TensorFlow backend.


In [15]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [16]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [17]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
############# Fully-Connected Neural Network ################

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [20]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [21]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 259 samples, validate on 112 samples
Epoch 1/1000
259/259 [==============================] - 0s 487us/step - loss: 10.9485 - accuracy: 0.4865 - val_loss: 0.9388 - val_accuracy: 0.5000
Epoch 2/1000
259/259 [==============================] - 0s 82us/step - loss: 0.7826 - accuracy: 0.5637 - val_loss: 0.8678 - val_accuracy: 0.5000
Epoch 3/1000
259/259 [==============================] - 0s 119us/step - loss: 0.7082 - accuracy: 0.5637 - val_loss: 0.6891 - val_accuracy: 0.5893
Epoch 4/1000
259/259 [==============================] - 0s 114us/step - loss: 0.6722 - accuracy: 0.6023 - val_loss: 0.6812 - val_accuracy: 0.5804
Epoch 5/1000
259/259 [==============================] - 0s 111us/step - loss: 0.6647 - accuracy: 0.5907 - val_loss: 0.6738 - val_accuracy: 0.5625
Epoch 6/1000
259/259 [==============================] - 0s 135us/step - loss: 0.6565 - accuracy: 0.5907 - val_loss: 0.6645 - val_accuracy: 0.5268
Epoch 7/1000
259/259 [==============================] - 0s 122us/step - loss: 

259/259 [==============================] - 0s 180us/step - loss: 0.5061 - accuracy: 0.7452 - val_loss: 0.5287 - val_accuracy: 0.7500
Epoch 57/1000
259/259 [==============================] - 0s 162us/step - loss: 0.5041 - accuracy: 0.7413 - val_loss: 0.5262 - val_accuracy: 0.7589
Epoch 58/1000
259/259 [==============================] - 0s 176us/step - loss: 0.5029 - accuracy: 0.7490 - val_loss: 0.5223 - val_accuracy: 0.7589
Epoch 59/1000
259/259 [==============================] - 0s 140us/step - loss: 0.4994 - accuracy: 0.7413 - val_loss: 0.5206 - val_accuracy: 0.7589
Epoch 60/1000
259/259 [==============================] - 0s 136us/step - loss: 0.4969 - accuracy: 0.7529 - val_loss: 0.5184 - val_accuracy: 0.7589
Epoch 61/1000
259/259 [==============================] - 0s 114us/step - loss: 0.4941 - accuracy: 0.7529 - val_loss: 0.5200 - val_accuracy: 0.7500
Epoch 62/1000
259/259 [==============================] - 0s 111us/step - loss: 0.4960 - accuracy: 0.7529 - val_loss: 0.5239 - val_ac

Epoch 112/1000
259/259 [==============================] - 0s 101us/step - loss: 0.4111 - accuracy: 0.7761 - val_loss: 0.4690 - val_accuracy: 0.7143
Epoch 113/1000
259/259 [==============================] - 0s 97us/step - loss: 0.4189 - accuracy: 0.7799 - val_loss: 0.4477 - val_accuracy: 0.7589
Epoch 114/1000
259/259 [==============================] - 0s 97us/step - loss: 0.4072 - accuracy: 0.7722 - val_loss: 0.4523 - val_accuracy: 0.7589
Epoch 115/1000
259/259 [==============================] - 0s 102us/step - loss: 0.4101 - accuracy: 0.7722 - val_loss: 0.4456 - val_accuracy: 0.7679
Epoch 116/1000
259/259 [==============================] - 0s 90us/step - loss: 0.4058 - accuracy: 0.7645 - val_loss: 0.4464 - val_accuracy: 0.7500
Epoch 117/1000
259/259 [==============================] - 0s 101us/step - loss: 0.4027 - accuracy: 0.7722 - val_loss: 0.4646 - val_accuracy: 0.7143
Epoch 118/1000
259/259 [==============================] - 0s 88us/step - loss: 0.4177 - accuracy: 0.7568 - val_loss

259/259 [==============================] - 0s 88us/step - loss: 0.3623 - accuracy: 0.8069 - val_loss: 0.4030 - val_accuracy: 0.8214
Epoch 168/1000
259/259 [==============================] - 0s 95us/step - loss: 0.3570 - accuracy: 0.8108 - val_loss: 0.4022 - val_accuracy: 0.7679
Epoch 169/1000
259/259 [==============================] - 0s 93us/step - loss: 0.3601 - accuracy: 0.8031 - val_loss: 0.3996 - val_accuracy: 0.7946
Epoch 170/1000
259/259 [==============================] - 0s 98us/step - loss: 0.3566 - accuracy: 0.7992 - val_loss: 0.4018 - val_accuracy: 0.8125
Epoch 171/1000
259/259 [==============================] - 0s 92us/step - loss: 0.3564 - accuracy: 0.7954 - val_loss: 0.4005 - val_accuracy: 0.8214
Epoch 172/1000
259/259 [==============================] - 0s 105us/step - loss: 0.3545 - accuracy: 0.8069 - val_loss: 0.4014 - val_accuracy: 0.7946
Epoch 173/1000
259/259 [==============================] - 0s 93us/step - loss: 0.3572 - accuracy: 0.8069 - val_loss: 0.3977 - val_ac

Epoch 223/1000
259/259 [==============================] - 0s 84us/step - loss: 0.3311 - accuracy: 0.8108 - val_loss: 0.3624 - val_accuracy: 0.8214
Epoch 224/1000
259/259 [==============================] - 0s 93us/step - loss: 0.3195 - accuracy: 0.8340 - val_loss: 0.3589 - val_accuracy: 0.8125
Epoch 225/1000
259/259 [==============================] - 0s 107us/step - loss: 0.3233 - accuracy: 0.8224 - val_loss: 0.3718 - val_accuracy: 0.8036
Epoch 226/1000
259/259 [==============================] - 0s 87us/step - loss: 0.3157 - accuracy: 0.8263 - val_loss: 0.3575 - val_accuracy: 0.8304
Epoch 227/1000
259/259 [==============================] - 0s 87us/step - loss: 0.3136 - accuracy: 0.8340 - val_loss: 0.3569 - val_accuracy: 0.8125
Epoch 228/1000
259/259 [==============================] - 0s 92us/step - loss: 0.3157 - accuracy: 0.8185 - val_loss: 0.3556 - val_accuracy: 0.8304
Epoch 229/1000
259/259 [==============================] - 0s 88us/step - loss: 0.3112 - accuracy: 0.8378 - val_loss: 

...

259/259 [==============================] - 0s 134us/step - loss: 0.2470 - accuracy: 0.8494 - val_loss: 0.3197 - val_accuracy: 0.8304
Epoch 776/1000
259/259 [==============================] - 0s 144us/step - loss: 0.2457 - accuracy: 0.8494 - val_loss: 0.3203 - val_accuracy: 0.8304
Epoch 777/1000
259/259 [==============================] - 0s 240us/step - loss: 0.2413 - accuracy: 0.8533 - val_loss: 0.3182 - val_accuracy: 0.8304
Epoch 778/1000
259/259 [==============================] - 0s 151us/step - loss: 0.2415 - accuracy: 0.8494 - val_loss: 0.3203 - val_accuracy: 0.8304
Epoch 779/1000
259/259 [==============================] - 0s 129us/step - loss: 0.2429 - accuracy: 0.8494 - val_loss: 0.3137 - val_accuracy: 0.8393
Epoch 780/1000
259/259 [==============================] - 0s 152us/step - loss: 0.2456 - accuracy: 0.8417 - val_loss: 0.3115 - val_accuracy: 0.8482
Epoch 781/1000
259/259 [==============================] - 0s 143us/step - loss: 0.2453 - accuracy: 0.8533 - val_loss: 0.3166 - 

Epoch 831/1000
259/259 [==============================] - 0s 114us/step - loss: 0.2389 - accuracy: 0.8533 - val_loss: 0.3161 - val_accuracy: 0.8482
Epoch 832/1000
259/259 [==============================] - 0s 94us/step - loss: 0.2372 - accuracy: 0.8610 - val_loss: 0.3204 - val_accuracy: 0.8304
Epoch 833/1000
259/259 [==============================] - 0s 94us/step - loss: 0.2393 - accuracy: 0.8494 - val_loss: 0.3145 - val_accuracy: 0.8482
Epoch 834/1000
259/259 [==============================] - 0s 98us/step - loss: 0.2366 - accuracy: 0.8571 - val_loss: 0.3162 - val_accuracy: 0.8393
Epoch 835/1000
259/259 [==============================] - 0s 91us/step - loss: 0.2376 - accuracy: 0.8571 - val_loss: 0.3230 - val_accuracy: 0.8304
Epoch 836/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2371 - accuracy: 0.8610 - val_loss: 0.3233 - val_accuracy: 0.8482
Epoch 837/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2389 - accuracy: 0.8610 - val_loss: 

Epoch 887/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2328 - accuracy: 0.8610 - val_loss: 0.3073 - val_accuracy: 0.8482
Epoch 888/1000
259/259 [==============================] - 0s 109us/step - loss: 0.2328 - accuracy: 0.8610 - val_loss: 0.3093 - val_accuracy: 0.8482
Epoch 889/1000
259/259 [==============================] - 0s 96us/step - loss: 0.2326 - accuracy: 0.8533 - val_loss: 0.3127 - val_accuracy: 0.8482
Epoch 890/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2321 - accuracy: 0.8610 - val_loss: 0.3077 - val_accuracy: 0.8482
Epoch 891/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2327 - accuracy: 0.8610 - val_loss: 0.3076 - val_accuracy: 0.8482
Epoch 892/1000
259/259 [==============================] - 0s 81us/step - loss: 0.2325 - accuracy: 0.8610 - val_loss: 0.3088 - val_accuracy: 0.8482
Epoch 893/1000
259/259 [==============================] - 0s 98us/step - loss: 0.2326 - accuracy: 0.8610 - val_loss: 

Epoch 943/1000
259/259 [==============================] - 0s 82us/step - loss: 0.2318 - accuracy: 0.8610 - val_loss: 0.3058 - val_accuracy: 0.8482
Epoch 944/1000
259/259 [==============================] - 0s 112us/step - loss: 0.2319 - accuracy: 0.8610 - val_loss: 0.3126 - val_accuracy: 0.8482
Epoch 945/1000
259/259 [==============================] - 0s 96us/step - loss: 0.2320 - accuracy: 0.8610 - val_loss: 0.3087 - val_accuracy: 0.8482
Epoch 946/1000
259/259 [==============================] - 0s 106us/step - loss: 0.2311 - accuracy: 0.8610 - val_loss: 0.3078 - val_accuracy: 0.8482
Epoch 947/1000
259/259 [==============================] - 0s 94us/step - loss: 0.2306 - accuracy: 0.8610 - val_loss: 0.3110 - val_accuracy: 0.8482
Epoch 948/1000
259/259 [==============================] - 0s 84us/step - loss: 0.2310 - accuracy: 0.8610 - val_loss: 0.3026 - val_accuracy: 0.8482
Epoch 949/1000
259/259 [==============================] - 0s 81us/step - loss: 0.2313 - accuracy: 0.8610 - val_loss:

259/259 [==============================] - 0s 120us/step - loss: 0.2293 - accuracy: 0.8610 - val_loss: 0.3059 - val_accuracy: 0.8393
Epoch 999/1000
259/259 [==============================] - 0s 129us/step - loss: 0.2306 - accuracy: 0.8610 - val_loss: 0.3056 - val_accuracy: 0.8482
Epoch 1000/1000
259/259 [==============================] - 0s 143us/step - loss: 0.2298 - accuracy: 0.8610 - val_loss: 0.3093 - val_accuracy: 0.8482


In [23]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

112/112 [==============================] - 0s 90us/step
combination test accuracy: 84.82%


In [47]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [48]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 259 samples, validate on 112 samples
Epoch 1/1000
259/259 [==============================] - 0s 458us/step - loss: 9.0329 - accuracy: 0.4981 - val_loss: 1.4286 - val_accuracy: 0.4554
Epoch 2/1000
259/259 [==============================] - 0s 97us/step - loss: 0.8933 - accuracy: 0.5598 - val_loss: 0.8746 - val_accuracy: 0.7054
Epoch 3/1000
259/259 [==============================] - 0s 127us/step - loss: 0.6689 - accuracy: 0.5676 - val_loss: 1.0205 - val_accuracy: 0.6786
Epoch 4/1000
259/259 [==============================] - 0s 114us/step - loss: 0.6956 - accuracy: 0.6641 - val_loss: 1.1800 - val_accuracy: 0.7232
Epoch 5/1000
259/259 [==============================] - 0s 126us/step - loss: 0.7113 - accuracy: 0.6757 - val_loss: 0.7995 - val_accuracy: 0.7143
Epoch 6/1000
259/259 [==============================] - 0s 141us/step - loss: 0.7706 - accuracy: 0.6216 - val_loss: 0.6844 - val_accuracy: 0.6250
Epoch 7/1000
259/259 [==============================] - 0s 117us/step - loss: 0

259/259 [==============================] - 0s 116us/step - loss: 0.4916 - accuracy: 0.7413 - val_loss: 0.5163 - val_accuracy: 0.7679
Epoch 57/1000
259/259 [==============================] - 0s 119us/step - loss: 0.4991 - accuracy: 0.7452 - val_loss: 0.5027 - val_accuracy: 0.7589
Epoch 58/1000
259/259 [==============================] - 0s 111us/step - loss: 0.4856 - accuracy: 0.7452 - val_loss: 0.5034 - val_accuracy: 0.7411
Epoch 59/1000
259/259 [==============================] - 0s 135us/step - loss: 0.4844 - accuracy: 0.7220 - val_loss: 0.4980 - val_accuracy: 0.7589
Epoch 60/1000
259/259 [==============================] - 0s 89us/step - loss: 0.4804 - accuracy: 0.7452 - val_loss: 0.4977 - val_accuracy: 0.7500
Epoch 61/1000
259/259 [==============================] - 0s 98us/step - loss: 0.4739 - accuracy: 0.7452 - val_loss: 0.5203 - val_accuracy: 0.6964
Epoch 62/1000
259/259 [==============================] - 0s 98us/step - loss: 0.4837 - accuracy: 0.7066 - val_loss: 0.4926 - val_accur

Epoch 112/1000
259/259 [==============================] - 0s 105us/step - loss: 0.3736 - accuracy: 0.7722 - val_loss: 0.4707 - val_accuracy: 0.7321
Epoch 113/1000
259/259 [==============================] - 0s 93us/step - loss: 0.3799 - accuracy: 0.7838 - val_loss: 0.4297 - val_accuracy: 0.7679
Epoch 114/1000
259/259 [==============================] - 0s 91us/step - loss: 0.3738 - accuracy: 0.7915 - val_loss: 0.4036 - val_accuracy: 0.7589
Epoch 115/1000
259/259 [==============================] - 0s 81us/step - loss: 0.3656 - accuracy: 0.7799 - val_loss: 0.4033 - val_accuracy: 0.8125
Epoch 116/1000
259/259 [==============================] - 0s 77us/step - loss: 0.3692 - accuracy: 0.7992 - val_loss: 0.3973 - val_accuracy: 0.8125
Epoch 117/1000
259/259 [==============================] - 0s 86us/step - loss: 0.3668 - accuracy: 0.7992 - val_loss: 0.4046 - val_accuracy: 0.7857
Epoch 118/1000
259/259 [==============================] - 0s 94us/step - loss: 0.3674 - accuracy: 0.7876 - val_loss: 

259/259 [==============================] - 0s 88us/step - loss: 0.3099 - accuracy: 0.8185 - val_loss: 0.3368 - val_accuracy: 0.8304
Epoch 168/1000
259/259 [==============================] - 0s 91us/step - loss: 0.3081 - accuracy: 0.8185 - val_loss: 0.3372 - val_accuracy: 0.8214
Epoch 169/1000
259/259 [==============================] - 0s 97us/step - loss: 0.3014 - accuracy: 0.8224 - val_loss: 0.3410 - val_accuracy: 0.8304
Epoch 170/1000
259/259 [==============================] - 0s 92us/step - loss: 0.3036 - accuracy: 0.8069 - val_loss: 0.3372 - val_accuracy: 0.7946
Epoch 171/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2992 - accuracy: 0.8301 - val_loss: 0.5649 - val_accuracy: 0.7589
Epoch 172/1000
259/259 [==============================] - 0s 165us/step - loss: 0.3720 - accuracy: 0.8069 - val_loss: 0.3657 - val_accuracy: 0.7679
Epoch 173/1000
259/259 [==============================] - 0s 96us/step - loss: 0.3306 - accuracy: 0.8108 - val_loss: 0.3411 - val_ac

259/259 [==============================] - 0s 88us/step - loss: 0.2898 - accuracy: 0.8417 - val_loss: 0.3208 - val_accuracy: 0.7857
Epoch 223/1000
259/259 [==============================] - 0s 91us/step - loss: 0.2732 - accuracy: 0.8417 - val_loss: 0.3137 - val_accuracy: 0.7946
Epoch 224/1000
259/259 [==============================] - 0s 86us/step - loss: 0.2680 - accuracy: 0.8417 - val_loss: 0.3074 - val_accuracy: 0.8036
Epoch 225/1000
259/259 [==============================] - 0s 99us/step - loss: 0.2618 - accuracy: 0.8610 - val_loss: 0.3710 - val_accuracy: 0.7857
Epoch 226/1000
259/259 [==============================] - 0s 94us/step - loss: 0.2934 - accuracy: 0.8456 - val_loss: 0.3660 - val_accuracy: 0.7679
Epoch 227/1000
259/259 [==============================] - 0s 94us/step - loss: 0.2863 - accuracy: 0.8263 - val_loss: 0.3180 - val_accuracy: 0.7857
Epoch 228/1000
259/259 [==============================] - 0s 99us/step - loss: 0.2709 - accuracy: 0.8456 - val_loss: 0.3412 - val_acc

...

Epoch 778/1000
259/259 [==============================] - 0s 90us/step - loss: 0.2035 - accuracy: 0.8764 - val_loss: 0.2638 - val_accuracy: 0.8214
Epoch 779/1000
259/259 [==============================] - 0s 95us/step - loss: 0.1913 - accuracy: 0.8842 - val_loss: 0.2587 - val_accuracy: 0.8304
Epoch 780/1000
259/259 [==============================] - 0s 100us/step - loss: 0.1958 - accuracy: 0.8803 - val_loss: 0.2651 - val_accuracy: 0.8304
Epoch 781/1000
259/259 [==============================] - 0s 88us/step - loss: 0.1859 - accuracy: 0.8880 - val_loss: 0.2658 - val_accuracy: 0.8304
Epoch 782/1000
259/259 [==============================] - 0s 99us/step - loss: 0.1895 - accuracy: 0.8842 - val_loss: 0.2821 - val_accuracy: 0.8125
Epoch 783/1000
259/259 [==============================] - 0s 91us/step - loss: 0.1844 - accuracy: 0.8880 - val_loss: 0.2796 - val_accuracy: 0.8125
Epoch 784/1000
259/259 [==============================] - 0s 152us/step - loss: 0.1871 - accuracy: 0.8880 - val_loss:

259/259 [==============================] - 0s 96us/step - loss: 0.2712 - accuracy: 0.8456 - val_loss: 0.6317 - val_accuracy: 0.7411
Epoch 834/1000
259/259 [==============================] - 0s 89us/step - loss: 0.5479 - accuracy: 0.7761 - val_loss: 0.4128 - val_accuracy: 0.7411
Epoch 835/1000
259/259 [==============================] - 0s 99us/step - loss: 0.3489 - accuracy: 0.7799 - val_loss: 0.3786 - val_accuracy: 0.7411
Epoch 836/1000
259/259 [==============================] - 0s 98us/step - loss: 0.2934 - accuracy: 0.8108 - val_loss: 0.3389 - val_accuracy: 0.7946
Epoch 837/1000
259/259 [==============================] - 0s 88us/step - loss: 0.2766 - accuracy: 0.8378 - val_loss: 0.3645 - val_accuracy: 0.7589
Epoch 838/1000
259/259 [==============================] - 0s 97us/step - loss: 0.2791 - accuracy: 0.8301 - val_loss: 0.3268 - val_accuracy: 0.7857
Epoch 839/1000
259/259 [==============================] - 0s 97us/step - loss: 0.2676 - accuracy: 0.8301 - val_loss: 0.3239 - val_acc

Epoch 889/1000
259/259 [==============================] - 0s 160us/step - loss: 0.1928 - accuracy: 0.8842 - val_loss: 0.2672 - val_accuracy: 0.8304
Epoch 890/1000
259/259 [==============================] - 0s 93us/step - loss: 0.2164 - accuracy: 0.8726 - val_loss: 0.3725 - val_accuracy: 0.8036
Epoch 891/1000
259/259 [==============================] - 0s 102us/step - loss: 0.1980 - accuracy: 0.8842 - val_loss: 0.7049 - val_accuracy: 0.8036
Epoch 892/1000
259/259 [==============================] - 0s 99us/step - loss: 0.3847 - accuracy: 0.8649 - val_loss: 0.2575 - val_accuracy: 0.8482
Epoch 893/1000
259/259 [==============================] - 0s 91us/step - loss: 0.1917 - accuracy: 0.8880 - val_loss: 0.2572 - val_accuracy: 0.8482
Epoch 894/1000
259/259 [==============================] - 0s 103us/step - loss: 0.1953 - accuracy: 0.8880 - val_loss: 0.2685 - val_accuracy: 0.8304
Epoch 895/1000
259/259 [==============================] - 0s 95us/step - loss: 0.1898 - accuracy: 0.8880 - val_loss

259/259 [==============================] - 0s 92us/step - loss: 0.1991 - accuracy: 0.8842 - val_loss: 0.2688 - val_accuracy: 0.8214
Epoch 945/1000
259/259 [==============================] - 0s 97us/step - loss: 0.1977 - accuracy: 0.8842 - val_loss: 0.2728 - val_accuracy: 0.8214
Epoch 946/1000
259/259 [==============================] - 0s 97us/step - loss: 0.1987 - accuracy: 0.8842 - val_loss: 0.2724 - val_accuracy: 0.8214
Epoch 947/1000
259/259 [==============================] - 0s 95us/step - loss: 0.1982 - accuracy: 0.8803 - val_loss: 0.2808 - val_accuracy: 0.8214
Epoch 948/1000
259/259 [==============================] - 0s 99us/step - loss: 0.1963 - accuracy: 0.8842 - val_loss: 0.2777 - val_accuracy: 0.8214
Epoch 949/1000
259/259 [==============================] - 0s 93us/step - loss: 0.1951 - accuracy: 0.8880 - val_loss: 0.5350 - val_accuracy: 0.8036
Epoch 950/1000
259/259 [==============================] - 0s 97us/step - loss: 0.2849 - accuracy: 0.8764 - val_loss: 0.2760 - val_acc

259/259 [==============================] - 0s 93us/step - loss: 0.1993 - accuracy: 0.8842 - val_loss: 0.2886 - val_accuracy: 0.8304
Epoch 1000/1000
259/259 [==============================] - 0s 94us/step - loss: 0.1949 - accuracy: 0.8880 - val_loss: 0.2746 - val_accuracy: 0.8482


In [50]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

112/112 [==============================] - 0s 90us/step
combination test accuracy: 84.82%


In [51]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [52]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 327us/step - loss: 8.2383 - accuracy: 0.5227 - val_loss: 0.6676 - val_accuracy: 0.5664
Epoch 2/1000
331/331 [==============================] - 0s 82us/step - loss: 0.6798 - accuracy: 0.5559 - val_loss: 0.6535 - val_accuracy: 0.6224
Epoch 3/1000
331/331 [==============================] - 0s 110us/step - loss: 0.6710 - accuracy: 0.5921 - val_loss: 0.6449 - val_accuracy: 0.6294
Epoch 4/1000
331/331 [==============================] - 0s 98us/step - loss: 0.6580 - accuracy: 0.6073 - val_loss: 0.6370 - val_accuracy: 0.7203
Epoch 5/1000
331/331 [==============================] - 0s 106us/step - loss: 0.6506 - accuracy: 0.6767 - val_loss: 0.6321 - val_accuracy: 0.7203
Epoch 6/1000
331/331 [==============================] - 0s 101us/step - loss: 0.6554 - accuracy: 0.6918 - val_loss: 0.6299 - val_accuracy: 0.7133
Epoch 7/1000
331/331 [==============================] - 0s 106us/step - loss: 0.

331/331 [==============================] - 0s 130us/step - loss: 0.5355 - accuracy: 0.7100 - val_loss: 0.5124 - val_accuracy: 0.6993
Epoch 57/1000
331/331 [==============================] - 0s 124us/step - loss: 0.5352 - accuracy: 0.7130 - val_loss: 0.5116 - val_accuracy: 0.6993
Epoch 58/1000
331/331 [==============================] - 0s 97us/step - loss: 0.5334 - accuracy: 0.7130 - val_loss: 0.5087 - val_accuracy: 0.6993
Epoch 59/1000
331/331 [==============================] - 0s 81us/step - loss: 0.5312 - accuracy: 0.7130 - val_loss: 0.5092 - val_accuracy: 0.6993
Epoch 60/1000
331/331 [==============================] - 0s 72us/step - loss: 0.5303 - accuracy: 0.7130 - val_loss: 0.5107 - val_accuracy: 0.7203
Epoch 61/1000
331/331 [==============================] - 0s 81us/step - loss: 0.5291 - accuracy: 0.7190 - val_loss: 0.5065 - val_accuracy: 0.7273
Epoch 62/1000
331/331 [==============================] - 0s 86us/step - loss: 0.5269 - accuracy: 0.7160 - val_loss: 0.5026 - val_accurac

331/331 [==============================] - 0s 82us/step - loss: 0.4731 - accuracy: 0.7130 - val_loss: 0.4549 - val_accuracy: 0.7413
Epoch 113/1000
331/331 [==============================] - 0s 87us/step - loss: 0.4732 - accuracy: 0.7311 - val_loss: 0.4530 - val_accuracy: 0.7483
Epoch 114/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4706 - accuracy: 0.7341 - val_loss: 0.4518 - val_accuracy: 0.7133
Epoch 115/1000
331/331 [==============================] - 0s 76us/step - loss: 0.4728 - accuracy: 0.7251 - val_loss: 0.4533 - val_accuracy: 0.7133
Epoch 116/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4702 - accuracy: 0.7432 - val_loss: 0.4542 - val_accuracy: 0.7133
Epoch 117/1000
331/331 [==============================] - 0s 85us/step - loss: 0.4697 - accuracy: 0.7251 - val_loss: 0.4494 - val_accuracy: 0.7133
Epoch 118/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4662 - accuracy: 0.7251 - val_loss: 0.4533 - val_acc

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.127881). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


331/331 [==============================] - 0s 90us/step - loss: 0.4618 - accuracy: 0.7583 - val_loss: 0.4387 - val_accuracy: 0.7483
Epoch 138/1000
331/331 [==============================] - 0s 94us/step - loss: 0.4576 - accuracy: 0.7644 - val_loss: 0.4374 - val_accuracy: 0.7203
Epoch 139/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4563 - accuracy: 0.7432 - val_loss: 0.4344 - val_accuracy: 0.7902
Epoch 140/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4556 - accuracy: 0.7402 - val_loss: 0.4409 - val_accuracy: 0.8182
Epoch 141/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4536 - accuracy: 0.7855 - val_loss: 0.4462 - val_accuracy: 0.7203
Epoch 142/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4546 - accuracy: 0.7644 - val_loss: 0.4319 - val_accuracy: 0.7972
Epoch 143/1000
331/331 [==============================] - 0s 84us/step - loss: 0.4509 - accuracy: 0.7764 - val_loss: 0.4309 - val_acc

...

Epoch 749/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3292 - accuracy: 0.8127 - val_loss: 0.3193 - val_accuracy: 0.8182
Epoch 750/1000
331/331 [==============================] - 0s 91us/step - loss: 0.3321 - accuracy: 0.8157 - val_loss: 0.3169 - val_accuracy: 0.8531
Epoch 751/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3272 - accuracy: 0.8218 - val_loss: 0.3134 - val_accuracy: 0.8322
Epoch 752/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3245 - accuracy: 0.8278 - val_loss: 0.3147 - val_accuracy: 0.8322
Epoch 753/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3275 - accuracy: 0.8218 - val_loss: 0.3078 - val_accuracy: 0.8392
Epoch 754/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3133 - accuracy: 0.8459 - val_loss: 0.2914 - val_accuracy: 0.8671
Epoch 755/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3039 - accuracy: 0.8550 - val_loss: 0

331/331 [==============================] - 0s 83us/step - loss: 0.3426 - accuracy: 0.8006 - val_loss: 0.3486 - val_accuracy: 0.7832
Epoch 805/1000
331/331 [==============================] - 0s 117us/step - loss: 0.3448 - accuracy: 0.7885 - val_loss: 0.3355 - val_accuracy: 0.8042
Epoch 806/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3416 - accuracy: 0.8066 - val_loss: 0.3325 - val_accuracy: 0.8252
Epoch 807/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3444 - accuracy: 0.8006 - val_loss: 0.3330 - val_accuracy: 0.8252
Epoch 808/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3460 - accuracy: 0.7976 - val_loss: 0.3344 - val_accuracy: 0.8182
Epoch 809/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3416 - accuracy: 0.8066 - val_loss: 0.3321 - val_accuracy: 0.7972
Epoch 810/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3414 - accuracy: 0.8066 - val_loss: 0.3446 - val_ac

Epoch 860/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3346 - accuracy: 0.8066 - val_loss: 0.3655 - val_accuracy: 0.7622
Epoch 861/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3405 - accuracy: 0.8036 - val_loss: 0.3247 - val_accuracy: 0.7972
Epoch 862/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3331 - accuracy: 0.8036 - val_loss: 0.3312 - val_accuracy: 0.8042
Epoch 863/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3393 - accuracy: 0.8036 - val_loss: 0.3314 - val_accuracy: 0.8252
Epoch 864/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3320 - accuracy: 0.8127 - val_loss: 0.3269 - val_accuracy: 0.7972
Epoch 865/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3339 - accuracy: 0.8066 - val_loss: 0.3384 - val_accuracy: 0.8112
Epoch 866/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3380 - accuracy: 0.7946 - val_loss: 0

331/331 [==============================] - 0s 78us/step - loss: 0.3270 - accuracy: 0.8127 - val_loss: 0.3194 - val_accuracy: 0.8042
Epoch 916/1000
331/331 [==============================] - 0s 95us/step - loss: 0.3269 - accuracy: 0.8066 - val_loss: 0.3160 - val_accuracy: 0.8322
Epoch 917/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3241 - accuracy: 0.8218 - val_loss: 0.3631 - val_accuracy: 0.7832
Epoch 918/1000
331/331 [==============================] - 0s 101us/step - loss: 0.3355 - accuracy: 0.8127 - val_loss: 0.3233 - val_accuracy: 0.8042
Epoch 919/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3232 - accuracy: 0.8187 - val_loss: 0.3191 - val_accuracy: 0.8042
Epoch 920/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3245 - accuracy: 0.8248 - val_loss: 0.3895 - val_accuracy: 0.7832
Epoch 921/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3335 - accuracy: 0.8157 - val_loss: 0.3244 - val_ac

Epoch 971/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3378 - accuracy: 0.8097 - val_loss: 0.3094 - val_accuracy: 0.8182
Epoch 972/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3191 - accuracy: 0.8157 - val_loss: 0.3171 - val_accuracy: 0.8392
Epoch 973/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3185 - accuracy: 0.8278 - val_loss: 0.3100 - val_accuracy: 0.8462
Epoch 974/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3151 - accuracy: 0.8338 - val_loss: 0.3132 - val_accuracy: 0.8182
Epoch 975/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3175 - accuracy: 0.8157 - val_loss: 0.3307 - val_accuracy: 0.8322
Epoch 976/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3222 - accuracy: 0.8187 - val_loss: 0.3059 - val_accuracy: 0.8182
Epoch 977/1000
331/331 [==============================] - 0s 125us/step - loss: 0.3170 - accuracy: 0.8218 - val_loss: 

In [54]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 72us/step
over-sampling test accuracy: 85.31%


In [55]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [56]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 386us/step - loss: 4.4713 - accuracy: 0.4924 - val_loss: 0.6877 - val_accuracy: 0.5315
Epoch 2/1000
331/331 [==============================] - 0s 105us/step - loss: 0.6774 - accuracy: 0.5378 - val_loss: 0.6762 - val_accuracy: 0.5315
Epoch 3/1000
331/331 [==============================] - 0s 101us/step - loss: 0.6607 - accuracy: 0.5498 - val_loss: 0.6680 - val_accuracy: 0.5245
Epoch 4/1000
331/331 [==============================] - 0s 96us/step - loss: 0.6579 - accuracy: 0.5498 - val_loss: 0.6724 - val_accuracy: 0.5315
Epoch 5/1000
331/331 [==============================] - 0s 110us/step - loss: 0.6522 - accuracy: 0.5619 - val_loss: 0.6622 - val_accuracy: 0.5245
Epoch 6/1000
331/331 [==============================] - 0s 129us/step - loss: 0.6467 - accuracy: 0.5619 - val_loss: 0.6544 - val_accuracy: 0.5245
Epoch 7/1000
331/331 [==============================] - 0s 111us/step - loss: 0

331/331 [==============================] - 0s 148us/step - loss: 0.5074 - accuracy: 0.7462 - val_loss: 0.5114 - val_accuracy: 0.7063
Epoch 57/1000
331/331 [==============================] - 0s 164us/step - loss: 0.5041 - accuracy: 0.7341 - val_loss: 0.4976 - val_accuracy: 0.7413
Epoch 58/1000
331/331 [==============================] - 0s 174us/step - loss: 0.5049 - accuracy: 0.7432 - val_loss: 0.4928 - val_accuracy: 0.7273
Epoch 59/1000
331/331 [==============================] - 0s 167us/step - loss: 0.4992 - accuracy: 0.7492 - val_loss: 0.4949 - val_accuracy: 0.7203
Epoch 60/1000
331/331 [==============================] - 0s 131us/step - loss: 0.4991 - accuracy: 0.7341 - val_loss: 0.5027 - val_accuracy: 0.7552
Epoch 61/1000
331/331 [==============================] - 0s 103us/step - loss: 0.5001 - accuracy: 0.7432 - val_loss: 0.4886 - val_accuracy: 0.7483
Epoch 62/1000
331/331 [==============================] - 0s 88us/step - loss: 0.4939 - accuracy: 0.7523 - val_loss: 0.4902 - val_acc

331/331 [==============================] - 0s 94us/step - loss: 0.4427 - accuracy: 0.7674 - val_loss: 0.4683 - val_accuracy: 0.7343
Epoch 113/1000
331/331 [==============================] - 0s 100us/step - loss: 0.4248 - accuracy: 0.7644 - val_loss: 0.4129 - val_accuracy: 0.7832
Epoch 114/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4193 - accuracy: 0.7825 - val_loss: 0.4419 - val_accuracy: 0.7622
Epoch 115/1000
331/331 [==============================] - 0s 89us/step - loss: 0.4187 - accuracy: 0.7885 - val_loss: 0.4175 - val_accuracy: 0.7622
Epoch 116/1000
331/331 [==============================] - 0s 79us/step - loss: 0.4159 - accuracy: 0.7976 - val_loss: 0.3993 - val_accuracy: 0.8112
Epoch 117/1000
331/331 [==============================] - 0s 85us/step - loss: 0.4203 - accuracy: 0.7825 - val_loss: 0.3906 - val_accuracy: 0.8042
Epoch 118/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4149 - accuracy: 0.7946 - val_loss: 0.4737 - val_ac

Epoch 168/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3846 - accuracy: 0.7915 - val_loss: 0.3480 - val_accuracy: 0.8322
Epoch 169/1000
331/331 [==============================] - 0s 94us/step - loss: 0.3834 - accuracy: 0.8006 - val_loss: 0.3703 - val_accuracy: 0.7762
Epoch 170/1000
331/331 [==============================] - 0s 108us/step - loss: 0.3883 - accuracy: 0.7704 - val_loss: 0.5248 - val_accuracy: 0.7273
Epoch 171/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3959 - accuracy: 0.7855 - val_loss: 0.3724 - val_accuracy: 0.7762
Epoch 172/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3761 - accuracy: 0.7885 - val_loss: 0.3635 - val_accuracy: 0.7762
Epoch 173/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3752 - accuracy: 0.7946 - val_loss: 0.6183 - val_accuracy: 0.7273
Epoch 174/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4208 - accuracy: 0.7764 - val_loss: 

Epoch 224/1000
331/331 [==============================] - 0s 145us/step - loss: 0.3423 - accuracy: 0.8097 - val_loss: 0.3096 - val_accuracy: 0.8322
Epoch 225/1000
331/331 [==============================] - 0s 91us/step - loss: 0.3519 - accuracy: 0.8066 - val_loss: 0.4058 - val_accuracy: 0.7832
Epoch 226/1000
331/331 [==============================] - 0s 100us/step - loss: 0.3578 - accuracy: 0.8006 - val_loss: 0.6059 - val_accuracy: 0.7832
Epoch 227/1000
331/331 [==============================] - 0s 106us/step - loss: 0.3833 - accuracy: 0.8066 - val_loss: 0.3265 - val_accuracy: 0.8112
Epoch 228/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3398 - accuracy: 0.8097 - val_loss: 0.3254 - val_accuracy: 0.7972
Epoch 229/1000
331/331 [==============================] - 0s 98us/step - loss: 0.3478 - accuracy: 0.8066 - val_loss: 0.3120 - val_accuracy: 0.8182
Epoch 230/1000
331/331 [==============================] - 0s 90us/step - loss: 0.3706 - accuracy: 0.7855 - val_loss

...

Epoch 726/1000
331/331 [==============================] - 0s 87us/step - loss: 0.2900 - accuracy: 0.8218 - val_loss: 0.2775 - val_accuracy: 0.8322
Epoch 727/1000
331/331 [==============================] - 0s 88us/step - loss: 0.2916 - accuracy: 0.8218 - val_loss: 0.4416 - val_accuracy: 0.7832
Epoch 728/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3119 - accuracy: 0.8157 - val_loss: 0.3013 - val_accuracy: 0.8252
Epoch 729/1000
331/331 [==============================] - 0s 82us/step - loss: 0.2936 - accuracy: 0.8127 - val_loss: 0.2660 - val_accuracy: 0.8392
Epoch 730/1000
331/331 [==============================] - 0s 73us/step - loss: 0.2885 - accuracy: 0.8218 - val_loss: 0.2678 - val_accuracy: 0.8252
Epoch 731/1000
331/331 [==============================] - 0s 83us/step - loss: 0.2851 - accuracy: 0.8248 - val_loss: 0.2644 - val_accuracy: 0.8392
Epoch 732/1000
331/331 [==============================] - 0s 82us/step - loss: 0.2899 - accuracy: 0.8248 - val_loss: 0

Epoch 782/1000
331/331 [==============================] - 0s 87us/step - loss: 0.2953 - accuracy: 0.8187 - val_loss: 0.3508 - val_accuracy: 0.7972
Epoch 783/1000
331/331 [==============================] - 0s 86us/step - loss: 0.2888 - accuracy: 0.8187 - val_loss: 0.4163 - val_accuracy: 0.7832
Epoch 784/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3199 - accuracy: 0.8127 - val_loss: 0.2865 - val_accuracy: 0.8252
Epoch 785/1000
331/331 [==============================] - 0s 82us/step - loss: 0.2901 - accuracy: 0.8218 - val_loss: 0.4187 - val_accuracy: 0.7832
Epoch 786/1000
331/331 [==============================] - 0s 84us/step - loss: 0.2957 - accuracy: 0.8157 - val_loss: 0.2764 - val_accuracy: 0.8252
Epoch 787/1000
331/331 [==============================] - 0s 92us/step - loss: 0.2844 - accuracy: 0.8218 - val_loss: 0.4439 - val_accuracy: 0.7832
Epoch 788/1000
331/331 [==============================] - 0s 80us/step - loss: 0.2931 - accuracy: 0.8218 - val_loss: 0

Epoch 838/1000
331/331 [==============================] - 0s 95us/step - loss: 0.2848 - accuracy: 0.8218 - val_loss: 0.2918 - val_accuracy: 0.8042
Epoch 839/1000
331/331 [==============================] - 0s 89us/step - loss: 0.2850 - accuracy: 0.8187 - val_loss: 0.2627 - val_accuracy: 0.8322
Epoch 840/1000
331/331 [==============================] - 0s 86us/step - loss: 0.2867 - accuracy: 0.8218 - val_loss: 0.2655 - val_accuracy: 0.8392
Epoch 841/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3147 - accuracy: 0.8006 - val_loss: 0.2744 - val_accuracy: 0.8252
Epoch 842/1000
331/331 [==============================] - 0s 85us/step - loss: 0.2865 - accuracy: 0.8157 - val_loss: 0.2987 - val_accuracy: 0.8252
Epoch 843/1000
331/331 [==============================] - 0s 86us/step - loss: 0.2901 - accuracy: 0.8187 - val_loss: 0.6273 - val_accuracy: 0.7552
Epoch 844/1000
331/331 [==============================] - 0s 120us/step - loss: 0.3045 - accuracy: 0.8157 - val_loss: 

331/331 [==============================] - 0s 119us/step - loss: 0.2820 - accuracy: 0.8278 - val_loss: 0.2726 - val_accuracy: 0.8252
Epoch 894/1000
331/331 [==============================] - 0s 104us/step - loss: 0.2780 - accuracy: 0.8248 - val_loss: 0.2625 - val_accuracy: 0.8392
Epoch 895/1000
331/331 [==============================] - 0s 111us/step - loss: 0.3089 - accuracy: 0.8097 - val_loss: 0.2702 - val_accuracy: 0.8252
Epoch 896/1000
331/331 [==============================] - 0s 106us/step - loss: 0.2828 - accuracy: 0.8187 - val_loss: 0.2736 - val_accuracy: 0.8252
Epoch 897/1000
331/331 [==============================] - 0s 104us/step - loss: 0.2846 - accuracy: 0.8187 - val_loss: 0.2968 - val_accuracy: 0.8252
Epoch 898/1000
331/331 [==============================] - 0s 97us/step - loss: 0.2873 - accuracy: 0.8218 - val_loss: 0.2655 - val_accuracy: 0.8392
Epoch 899/1000
331/331 [==============================] - 0s 98us/step - loss: 0.2789 - accuracy: 0.8248 - val_loss: 0.2698 - va

331/331 [==============================] - 0s 108us/step - loss: 0.2915 - accuracy: 0.8127 - val_loss: 0.2667 - val_accuracy: 0.8392
Epoch 949/1000
331/331 [==============================] - 0s 111us/step - loss: 0.2791 - accuracy: 0.8248 - val_loss: 0.2756 - val_accuracy: 0.8252
Epoch 950/1000
331/331 [==============================] - 0s 108us/step - loss: 0.2781 - accuracy: 0.8218 - val_loss: 0.2661 - val_accuracy: 0.8392
Epoch 951/1000
331/331 [==============================] - 0s 92us/step - loss: 0.2765 - accuracy: 0.8278 - val_loss: 0.2683 - val_accuracy: 0.8322
Epoch 952/1000
331/331 [==============================] - 0s 99us/step - loss: 0.3223 - accuracy: 0.7946 - val_loss: 1.3827 - val_accuracy: 0.7273
Epoch 953/1000
331/331 [==============================] - 0s 113us/step - loss: 0.4459 - accuracy: 0.8127 - val_loss: 0.2774 - val_accuracy: 0.8252
Epoch 954/1000
331/331 [==============================] - 0s 80us/step - loss: 0.2788 - accuracy: 0.8248 - val_loss: 0.2683 - val

In [58]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 78us/step
over-sampling test accuracy: 83.22%


In [59]:
############ Feature selection using lasso ##########

In [60]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [61]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [62]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [63]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [64]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 146
selected features: 17


In [65]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  2,   3,   5,  11,  34,  45,  59,  79,  87, 103, 104, 105, 106,
        109, 113, 144, 145]])

In [66]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTAATACATAT', 'TTTTAAATATTATAA', 'TTTATCTTTATGA',
       'TTCAAGAAGGAGA', 'TCTCCTTCTTGA', 'TACAACGTCTT', 'GGAATCTTACT',
       'CTGGATCCG', 'CAATGACTATTAT', 'ribN', 'purC', 'group_5797',
       'group_8402', 'group_3457', 'group_562', 'ST', 'CC'], dtype='<U100')

In [67]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   4,   6,   7,   8,   9,  10,  12,  13,
            ...
            134, 135, 136, 137, 138, 139, 140, 141, 142, 143],
           dtype='int64', length=129)

In [68]:
len(removed_features)

129

In [69]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 17) (255,) (255, 18)


In [70]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 201), (1, 169)]


In [71]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 237), (1, 237)]


In [72]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [73]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [74]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [75]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [76]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 259 samples, validate on 111 samples
Epoch 1/1000
259/259 [==============================] - 0s 566us/step - loss: 7.5410 - accuracy: 0.4903 - val_loss: 0.7362 - val_accuracy: 0.4685
Epoch 2/1000
259/259 [==============================] - 0s 84us/step - loss: 0.7262 - accuracy: 0.4672 - val_loss: 0.7224 - val_accuracy: 0.5135
Epoch 3/1000
259/259 [==============================] - 0s 115us/step - loss: 0.7222 - accuracy: 0.4865 - val_loss: 0.7148 - val_accuracy: 0.3964
Epoch 4/1000
259/259 [==============================] - 0s 113us/step - loss: 0.7097 - accuracy: 0.4131 - val_loss: 0.7036 - val_accuracy: 0.4414
Epoch 5/1000
259/259 [==============================] - 0s 115us/step - loss: 0.7031 - accuracy: 0.4479 - val_loss: 0.7378 - val_accuracy: 0.5405
Epoch 6/1000
259/259 [==============================] - 0s 137us/step - loss: 0.7155 - accuracy: 0.4440 - val_loss: 0.7174 - val_accuracy: 0.4865
Epoch 7/1000
259/259 [==============================] - 0s 117us/step - loss: 0

259/259 [==============================] - 0s 162us/step - loss: 0.5994 - accuracy: 0.6950 - val_loss: 0.6064 - val_accuracy: 0.7387
Epoch 57/1000
259/259 [==============================] - 0s 135us/step - loss: 0.5959 - accuracy: 0.6911 - val_loss: 0.6039 - val_accuracy: 0.8018
Epoch 58/1000
259/259 [==============================] - 0s 139us/step - loss: 0.5946 - accuracy: 0.7297 - val_loss: 0.5903 - val_accuracy: 0.8468
Epoch 59/1000
259/259 [==============================] - 0s 126us/step - loss: 0.5887 - accuracy: 0.7722 - val_loss: 0.5926 - val_accuracy: 0.7297
Epoch 60/1000
259/259 [==============================] - 0s 116us/step - loss: 0.5864 - accuracy: 0.7336 - val_loss: 0.5937 - val_accuracy: 0.7477
Epoch 61/1000
259/259 [==============================] - 0s 98us/step - loss: 0.5885 - accuracy: 0.7297 - val_loss: 0.5848 - val_accuracy: 0.8108
Epoch 62/1000
259/259 [==============================] - 0s 110us/step - loss: 0.5848 - accuracy: 0.7799 - val_loss: 0.5967 - val_acc

259/259 [==============================] - 0s 89us/step - loss: 0.5355 - accuracy: 0.7954 - val_loss: 0.5313 - val_accuracy: 0.7568
Epoch 113/1000
259/259 [==============================] - 0s 89us/step - loss: 0.5160 - accuracy: 0.8108 - val_loss: 0.4972 - val_accuracy: 0.8739
Epoch 114/1000
259/259 [==============================] - 0s 85us/step - loss: 0.4942 - accuracy: 0.8301 - val_loss: 0.4923 - val_accuracy: 0.8559
Epoch 115/1000
259/259 [==============================] - 0s 89us/step - loss: 0.5004 - accuracy: 0.8224 - val_loss: 0.4949 - val_accuracy: 0.8739
Epoch 116/1000
259/259 [==============================] - 0s 88us/step - loss: 0.5071 - accuracy: 0.8147 - val_loss: 0.7364 - val_accuracy: 0.7117
Epoch 117/1000
259/259 [==============================] - 0s 96us/step - loss: 0.5365 - accuracy: 0.7954 - val_loss: 0.5184 - val_accuracy: 0.8018
Epoch 118/1000
259/259 [==============================] - 0s 100us/step - loss: 0.4869 - accuracy: 0.8378 - val_loss: 0.4908 - val_ac

Epoch 168/1000
259/259 [==============================] - 0s 90us/step - loss: 0.4439 - accuracy: 0.8263 - val_loss: 0.4208 - val_accuracy: 0.8829
Epoch 169/1000
259/259 [==============================] - 0s 90us/step - loss: 0.4427 - accuracy: 0.8108 - val_loss: 0.4202 - val_accuracy: 0.8829
Epoch 170/1000
259/259 [==============================] - 0s 91us/step - loss: 0.4340 - accuracy: 0.8417 - val_loss: 0.4244 - val_accuracy: 0.8559
Epoch 171/1000
259/259 [==============================] - 0s 84us/step - loss: 0.4288 - accuracy: 0.8417 - val_loss: 0.4153 - val_accuracy: 0.8739
Epoch 172/1000
259/259 [==============================] - 0s 87us/step - loss: 0.4280 - accuracy: 0.8340 - val_loss: 0.4189 - val_accuracy: 0.8829
Epoch 173/1000
259/259 [==============================] - 0s 96us/step - loss: 0.4214 - accuracy: 0.8494 - val_loss: 0.4229 - val_accuracy: 0.8829
Epoch 174/1000
259/259 [==============================] - 0s 121us/step - loss: 0.4418 - accuracy: 0.8224 - val_loss: 

259/259 [==============================] - 0s 82us/step - loss: 0.4122 - accuracy: 0.8301 - val_loss: 0.4216 - val_accuracy: 0.8649
Epoch 224/1000
259/259 [==============================] - 0s 95us/step - loss: 0.4015 - accuracy: 0.8340 - val_loss: 0.4747 - val_accuracy: 0.8378
Epoch 225/1000
259/259 [==============================] - 0s 136us/step - loss: 0.4148 - accuracy: 0.8301 - val_loss: 0.3911 - val_accuracy: 0.8649
Epoch 226/1000
259/259 [==============================] - 0s 120us/step - loss: 0.4023 - accuracy: 0.8340 - val_loss: 0.4157 - val_accuracy: 0.8649
Epoch 227/1000
259/259 [==============================] - 0s 128us/step - loss: 0.4154 - accuracy: 0.8301 - val_loss: 0.3946 - val_accuracy: 0.8559
Epoch 228/1000
259/259 [==============================] - 0s 128us/step - loss: 0.4061 - accuracy: 0.8224 - val_loss: 0.3916 - val_accuracy: 0.8559
Epoch 229/1000
259/259 [==============================] - 0s 89us/step - loss: 0.3951 - accuracy: 0.8301 - val_loss: 0.3894 - val

...

259/259 [==============================] - 0s 115us/step - loss: 0.2836 - accuracy: 0.8533 - val_loss: 0.2728 - val_accuracy: 0.8739
Epoch 724/1000
259/259 [==============================] - 0s 121us/step - loss: 0.2796 - accuracy: 0.8533 - val_loss: 0.2786 - val_accuracy: 0.8739
Epoch 725/1000
259/259 [==============================] - 0s 106us/step - loss: 0.2828 - accuracy: 0.8417 - val_loss: 0.2746 - val_accuracy: 0.8739
Epoch 726/1000
259/259 [==============================] - 0s 139us/step - loss: 0.2893 - accuracy: 0.8378 - val_loss: 0.2904 - val_accuracy: 0.8649
Epoch 727/1000
259/259 [==============================] - 0s 117us/step - loss: 0.2839 - accuracy: 0.8571 - val_loss: 0.2800 - val_accuracy: 0.9009
Epoch 728/1000
259/259 [==============================] - 0s 127us/step - loss: 0.2826 - accuracy: 0.8494 - val_loss: 0.2740 - val_accuracy: 0.8829
Epoch 729/1000
259/259 [==============================] - 0s 142us/step - loss: 0.2889 - accuracy: 0.8494 - val_loss: 0.3120 - 

259/259 [==============================] - 0s 113us/step - loss: 0.2804 - accuracy: 0.8301 - val_loss: 0.2984 - val_accuracy: 0.8559
Epoch 779/1000
259/259 [==============================] - 0s 93us/step - loss: 0.2807 - accuracy: 0.8378 - val_loss: 0.2776 - val_accuracy: 0.8468
Epoch 780/1000
259/259 [==============================] - 0s 84us/step - loss: 0.2846 - accuracy: 0.8378 - val_loss: 0.2852 - val_accuracy: 0.8468
Epoch 781/1000
259/259 [==============================] - 0s 96us/step - loss: 0.2755 - accuracy: 0.8378 - val_loss: 0.3091 - val_accuracy: 0.8378
Epoch 782/1000
259/259 [==============================] - 0s 86us/step - loss: 0.2885 - accuracy: 0.8378 - val_loss: 0.3029 - val_accuracy: 0.8198
Epoch 783/1000
259/259 [==============================] - 0s 89us/step - loss: 0.2780 - accuracy: 0.8378 - val_loss: 0.2766 - val_accuracy: 0.8468
Epoch 784/1000
259/259 [==============================] - 0s 90us/step - loss: 0.2800 - accuracy: 0.8378 - val_loss: 0.2898 - val_ac

Epoch 834/1000
259/259 [==============================] - 0s 88us/step - loss: 0.2649 - accuracy: 0.8378 - val_loss: 0.3968 - val_accuracy: 0.8288
Epoch 835/1000
259/259 [==============================] - 0s 108us/step - loss: 0.3178 - accuracy: 0.8378 - val_loss: 0.3628 - val_accuracy: 0.8288
Epoch 836/1000
259/259 [==============================] - 0s 90us/step - loss: 0.3283 - accuracy: 0.8378 - val_loss: 0.2769 - val_accuracy: 0.8378
Epoch 837/1000
259/259 [==============================] - 0s 90us/step - loss: 0.2802 - accuracy: 0.8378 - val_loss: 0.2734 - val_accuracy: 0.8468
Epoch 838/1000
259/259 [==============================] - 0s 85us/step - loss: 0.2692 - accuracy: 0.8378 - val_loss: 0.2794 - val_accuracy: 0.8468
Epoch 839/1000
259/259 [==============================] - 0s 95us/step - loss: 0.2674 - accuracy: 0.8378 - val_loss: 0.2814 - val_accuracy: 0.8468
Epoch 840/1000
259/259 [==============================] - 0s 84us/step - loss: 0.2783 - accuracy: 0.8378 - val_loss: 

Epoch 890/1000
259/259 [==============================] - 0s 90us/step - loss: 0.3101 - accuracy: 0.8378 - val_loss: 0.3000 - val_accuracy: 0.8378
Epoch 891/1000
259/259 [==============================] - 0s 102us/step - loss: 0.3101 - accuracy: 0.8378 - val_loss: 0.3003 - val_accuracy: 0.8378
Epoch 892/1000
259/259 [==============================] - 0s 95us/step - loss: 0.3100 - accuracy: 0.8378 - val_loss: 0.3134 - val_accuracy: 0.8288
Epoch 893/1000
259/259 [==============================] - 0s 94us/step - loss: 0.3125 - accuracy: 0.8378 - val_loss: 0.3006 - val_accuracy: 0.8378
Epoch 894/1000
259/259 [==============================] - 0s 89us/step - loss: 0.3096 - accuracy: 0.8378 - val_loss: 0.3009 - val_accuracy: 0.8378
Epoch 895/1000
259/259 [==============================] - 0s 100us/step - loss: 0.3094 - accuracy: 0.8378 - val_loss: 0.2998 - val_accuracy: 0.8378
Epoch 896/1000
259/259 [==============================] - 0s 90us/step - loss: 0.3087 - accuracy: 0.8378 - val_loss:

259/259 [==============================] - 0s 84us/step - loss: 0.2750 - accuracy: 0.8378 - val_loss: 0.2650 - val_accuracy: 0.8559
Epoch 946/1000
259/259 [==============================] - 0s 90us/step - loss: 0.2682 - accuracy: 0.8378 - val_loss: 0.2678 - val_accuracy: 0.8559
Epoch 947/1000
259/259 [==============================] - 0s 95us/step - loss: 0.2651 - accuracy: 0.8378 - val_loss: 0.2639 - val_accuracy: 0.8559
Epoch 948/1000
259/259 [==============================] - 0s 104us/step - loss: 0.2765 - accuracy: 0.8378 - val_loss: 0.2648 - val_accuracy: 0.8559
Epoch 949/1000
259/259 [==============================] - 0s 104us/step - loss: 0.2696 - accuracy: 0.8378 - val_loss: 0.2631 - val_accuracy: 0.8559
Epoch 950/1000
259/259 [==============================] - 0s 84us/step - loss: 0.2688 - accuracy: 0.8378 - val_loss: 0.2748 - val_accuracy: 0.8468
Epoch 951/1000
259/259 [==============================] - 0s 94us/step - loss: 0.2602 - accuracy: 0.8378 - val_loss: 0.2820 - val_a

In [77]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


111/111 [==============================] - 0s 122us/step
combination test accuracy: 84.68%


In [99]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [100]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [101]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 259 samples, validate on 111 samples
Epoch 1/1000
259/259 [==============================] - 0s 451us/step - loss: 11.5111 - accuracy: 0.4826 - val_loss: 1.9371 - val_accuracy: 0.5586
Epoch 2/1000
259/259 [==============================] - 0s 52us/step - loss: 2.4773 - accuracy: 0.4710 - val_loss: 0.7912 - val_accuracy: 0.4865
Epoch 3/1000
259/259 [==============================] - 0s 80us/step - loss: 0.7634 - accuracy: 0.5290 - val_loss: 0.7875 - val_accuracy: 0.5856
Epoch 4/1000
259/259 [==============================] - 0s 82us/step - loss: 0.8792 - accuracy: 0.6100 - val_loss: 1.0606 - val_accuracy: 0.5225
Epoch 5/1000
259/259 [==============================] - 0s 85us/step - loss: 0.9127 - accuracy: 0.5058 - val_loss: 0.7139 - val_accuracy: 0.5045
Epoch 6/1000
259/259 [==============================] - 0s 74us/step - loss: 0.7200 - accuracy: 0.4942 - val_loss: 0.7778 - val_accuracy: 0.5225
Epoch 7/1000
259/259 [==============================] - 0s 86us/step - loss: 0.726

Epoch 57/1000
259/259 [==============================] - 0s 83us/step - loss: 0.6322 - accuracy: 0.7683 - val_loss: 0.7476 - val_accuracy: 0.6667
Epoch 58/1000
259/259 [==============================] - 0s 96us/step - loss: 0.6909 - accuracy: 0.6525 - val_loss: 0.6215 - val_accuracy: 0.7477
Epoch 59/1000
259/259 [==============================] - 0s 72us/step - loss: 0.6145 - accuracy: 0.7606 - val_loss: 0.6203 - val_accuracy: 0.7477
Epoch 60/1000
259/259 [==============================] - 0s 162us/step - loss: 0.6126 - accuracy: 0.7066 - val_loss: 0.6223 - val_accuracy: 0.8288
Epoch 61/1000
259/259 [==============================] - 0s 71us/step - loss: 0.6165 - accuracy: 0.7645 - val_loss: 0.6197 - val_accuracy: 0.7477
Epoch 62/1000
259/259 [==============================] - 0s 67us/step - loss: 0.6120 - accuracy: 0.7220 - val_loss: 0.6157 - val_accuracy: 0.7568
Epoch 63/1000
259/259 [==============================] - 0s 60us/step - loss: 0.6114 - accuracy: 0.7259 - val_loss: 0.6178 

Epoch 113/1000
259/259 [==============================] - 0s 97us/step - loss: 0.5548 - accuracy: 0.7761 - val_loss: 0.5641 - val_accuracy: 0.8198
Epoch 114/1000
259/259 [==============================] - 0s 61us/step - loss: 0.5471 - accuracy: 0.8417 - val_loss: 0.5746 - val_accuracy: 0.7387
Epoch 115/1000
259/259 [==============================] - 0s 157us/step - loss: 0.5661 - accuracy: 0.7954 - val_loss: 0.5573 - val_accuracy: 0.8468
Epoch 116/1000
259/259 [==============================] - 0s 59us/step - loss: 0.5458 - accuracy: 0.8301 - val_loss: 0.5517 - val_accuracy: 0.8198
Epoch 117/1000
259/259 [==============================] - 0s 65us/step - loss: 0.5512 - accuracy: 0.8108 - val_loss: 0.5656 - val_accuracy: 0.8018
Epoch 118/1000
259/259 [==============================] - 0s 56us/step - loss: 0.5516 - accuracy: 0.8185 - val_loss: 0.5528 - val_accuracy: 0.8198
Epoch 119/1000
259/259 [==============================] - 0s 64us/step - loss: 0.5427 - accuracy: 0.8224 - val_loss: 

Epoch 169/1000
259/259 [==============================] - 0s 68us/step - loss: 0.4853 - accuracy: 0.8224 - val_loss: 0.5859 - val_accuracy: 0.7477
Epoch 170/1000
259/259 [==============================] - 0s 62us/step - loss: 0.5145 - accuracy: 0.7876 - val_loss: 0.5566 - val_accuracy: 0.7658
Epoch 171/1000
259/259 [==============================] - 0s 53us/step - loss: 0.4967 - accuracy: 0.7954 - val_loss: 0.4990 - val_accuracy: 0.7748
Epoch 172/1000
259/259 [==============================] - 0s 62us/step - loss: 0.4756 - accuracy: 0.8147 - val_loss: 0.4786 - val_accuracy: 0.7928
Epoch 173/1000
259/259 [==============================] - 0s 51us/step - loss: 0.4704 - accuracy: 0.8147 - val_loss: 0.4859 - val_accuracy: 0.8018
Epoch 174/1000
259/259 [==============================] - 0s 50us/step - loss: 0.4761 - accuracy: 0.8185 - val_loss: 0.4919 - val_accuracy: 0.8468
Epoch 175/1000
259/259 [==============================] - 0s 59us/step - loss: 0.4734 - accuracy: 0.8147 - val_loss: 1

Epoch 225/1000
259/259 [==============================] - 0s 64us/step - loss: 0.4103 - accuracy: 0.8301 - val_loss: 0.6666 - val_accuracy: 0.7117
Epoch 226/1000
259/259 [==============================] - 0s 81us/step - loss: 0.4918 - accuracy: 0.7992 - val_loss: 0.8910 - val_accuracy: 0.6937
Epoch 227/1000
259/259 [==============================] - 0s 66us/step - loss: 0.5978 - accuracy: 0.8031 - val_loss: 0.5266 - val_accuracy: 0.8559
Epoch 228/1000
259/259 [==============================] - 0s 64us/step - loss: 0.5224 - accuracy: 0.8108 - val_loss: 0.4232 - val_accuracy: 0.8468
Epoch 229/1000
259/259 [==============================] - 0s 61us/step - loss: 0.4517 - accuracy: 0.7954 - val_loss: 0.4523 - val_accuracy: 0.8649
Epoch 230/1000
259/259 [==============================] - 0s 71us/step - loss: 0.4179 - accuracy: 0.8417 - val_loss: 0.5689 - val_accuracy: 0.8468
Epoch 231/1000
259/259 [==============================] - 0s 63us/step - loss: 0.4535 - accuracy: 0.8263 - val_loss: 1

...

259/259 [==============================] - 0s 68us/step - loss: 0.3106 - accuracy: 0.8301 - val_loss: 0.3266 - val_accuracy: 0.8378
Epoch 726/1000
259/259 [==============================] - 0s 63us/step - loss: 0.3109 - accuracy: 0.8224 - val_loss: 0.3273 - val_accuracy: 0.8198
Epoch 727/1000
259/259 [==============================] - 0s 68us/step - loss: 0.3138 - accuracy: 0.8263 - val_loss: 0.3264 - val_accuracy: 0.8198
Epoch 728/1000
259/259 [==============================] - 0s 71us/step - loss: 0.3117 - accuracy: 0.8263 - val_loss: 0.3211 - val_accuracy: 0.8198
Epoch 729/1000
259/259 [==============================] - 0s 65us/step - loss: 0.2937 - accuracy: 0.8224 - val_loss: 0.4093 - val_accuracy: 0.8198
Epoch 730/1000
259/259 [==============================] - 0s 68us/step - loss: 0.3247 - accuracy: 0.8263 - val_loss: 0.3220 - val_accuracy: 0.8198
Epoch 731/1000
259/259 [==============================] - 0s 75us/step - loss: 0.2872 - accuracy: 0.8301 - val_loss: 0.3810 - val_acc

259/259 [==============================] - 0s 65us/step - loss: 0.2981 - accuracy: 0.8108 - val_loss: 0.3079 - val_accuracy: 0.8559
Epoch 781/1000
259/259 [==============================] - 0s 62us/step - loss: 0.2788 - accuracy: 0.8456 - val_loss: 0.3157 - val_accuracy: 0.8288
Epoch 782/1000
259/259 [==============================] - 0s 81us/step - loss: 0.2770 - accuracy: 0.8340 - val_loss: 0.3123 - val_accuracy: 0.8288
Epoch 783/1000
259/259 [==============================] - 0s 64us/step - loss: 0.2821 - accuracy: 0.8340 - val_loss: 0.3102 - val_accuracy: 0.8198
Epoch 784/1000
259/259 [==============================] - 0s 73us/step - loss: 0.2816 - accuracy: 0.8378 - val_loss: 0.3113 - val_accuracy: 0.8288
Epoch 785/1000
259/259 [==============================] - 0s 77us/step - loss: 0.2764 - accuracy: 0.8494 - val_loss: 0.3244 - val_accuracy: 0.8288
Epoch 786/1000
259/259 [==============================] - 0s 66us/step - loss: 0.2864 - accuracy: 0.8301 - val_loss: 0.3071 - val_acc

Epoch 836/1000
259/259 [==============================] - 0s 71us/step - loss: 0.4238 - accuracy: 0.8417 - val_loss: 0.3068 - val_accuracy: 0.8288
Epoch 837/1000
259/259 [==============================] - 0s 65us/step - loss: 0.2781 - accuracy: 0.8456 - val_loss: 0.3176 - val_accuracy: 0.8288
Epoch 838/1000
259/259 [==============================] - 0s 70us/step - loss: 0.2793 - accuracy: 0.8417 - val_loss: 0.3056 - val_accuracy: 0.8288
Epoch 839/1000
259/259 [==============================] - 0s 74us/step - loss: 0.2706 - accuracy: 0.8340 - val_loss: 0.4566 - val_accuracy: 0.7117
Epoch 840/1000
259/259 [==============================] - 0s 65us/step - loss: 0.3655 - accuracy: 0.7413 - val_loss: 0.3369 - val_accuracy: 0.8198
Epoch 841/1000
259/259 [==============================] - 0s 69us/step - loss: 0.3161 - accuracy: 0.8456 - val_loss: 0.3191 - val_accuracy: 0.8288
Epoch 842/1000
259/259 [==============================] - 0s 73us/step - loss: 0.3044 - accuracy: 0.8378 - val_loss: 0

Epoch 892/1000
259/259 [==============================] - 0s 46us/step - loss: 0.2862 - accuracy: 0.8340 - val_loss: 0.2950 - val_accuracy: 0.8468
Epoch 893/1000
259/259 [==============================] - 0s 67us/step - loss: 0.2737 - accuracy: 0.8494 - val_loss: 0.2943 - val_accuracy: 0.8559
Epoch 894/1000
259/259 [==============================] - 0s 66us/step - loss: 0.2716 - accuracy: 0.8494 - val_loss: 0.2920 - val_accuracy: 0.8559
Epoch 895/1000
259/259 [==============================] - 0s 189us/step - loss: 0.2701 - accuracy: 0.8494 - val_loss: 0.2948 - val_accuracy: 0.8559
Epoch 896/1000
259/259 [==============================] - 0s 68us/step - loss: 0.2693 - accuracy: 0.8494 - val_loss: 0.3510 - val_accuracy: 0.8288
Epoch 897/1000
259/259 [==============================] - 0s 63us/step - loss: 0.2976 - accuracy: 0.8378 - val_loss: 0.2992 - val_accuracy: 0.8559
Epoch 898/1000
259/259 [==============================] - 0s 68us/step - loss: 0.2784 - accuracy: 0.8494 - val_loss: 

259/259 [==============================] - 0s 65us/step - loss: 0.2727 - accuracy: 0.8456 - val_loss: 0.2864 - val_accuracy: 0.8649
Epoch 948/1000
259/259 [==============================] - 0s 84us/step - loss: 0.2673 - accuracy: 0.8456 - val_loss: 0.2832 - val_accuracy: 0.8739
Epoch 949/1000
259/259 [==============================] - 0s 90us/step - loss: 0.2743 - accuracy: 0.8494 - val_loss: 0.2874 - val_accuracy: 0.8739
Epoch 950/1000
259/259 [==============================] - 0s 83us/step - loss: 0.2684 - accuracy: 0.8494 - val_loss: 0.3049 - val_accuracy: 0.8649
Epoch 951/1000
259/259 [==============================] - 0s 80us/step - loss: 0.2658 - accuracy: 0.8533 - val_loss: 0.3055 - val_accuracy: 0.8559
Epoch 952/1000
259/259 [==============================] - 0s 121us/step - loss: 0.2663 - accuracy: 0.8533 - val_loss: 0.8640 - val_accuracy: 0.8108
Epoch 953/1000
259/259 [==============================] - 0s 76us/step - loss: 0.4370 - accuracy: 0.8456 - val_loss: 0.4429 - val_ac

In [102]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

111/111 [==============================] - 0s 93us/step
combination test accuracy: 87.39%


In [103]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [104]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [105]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 325us/step - loss: 23.4328 - accuracy: 0.4502 - val_loss: 0.7128 - val_accuracy: 0.5315
Epoch 2/1000
331/331 [==============================] - 0s 82us/step - loss: 0.7150 - accuracy: 0.4653 - val_loss: 0.6978 - val_accuracy: 0.5105
Epoch 3/1000
331/331 [==============================] - 0s 104us/step - loss: 0.7131 - accuracy: 0.4713 - val_loss: 0.6990 - val_accuracy: 0.4615
Epoch 4/1000
331/331 [==============================] - 0s 105us/step - loss: 0.7101 - accuracy: 0.4834 - val_loss: 0.7322 - val_accuracy: 0.4196
Epoch 5/1000
331/331 [==============================] - 0s 103us/step - loss: 0.7152 - accuracy: 0.4320 - val_loss: 0.6933 - val_accuracy: 0.5385
Epoch 6/1000
331/331 [==============================] - 0s 98us/step - loss: 0.7060 - accuracy: 0.5045 - val_loss: 0.7357 - val_accuracy: 0.4266
Epoch 7/1000
331/331 [==============================] - 0s 86us/step - loss: 0.

331/331 [==============================] - 0s 100us/step - loss: 0.6717 - accuracy: 0.5740 - val_loss: 0.6703 - val_accuracy: 0.6573
Epoch 57/1000
331/331 [==============================] - 0s 105us/step - loss: 0.6702 - accuracy: 0.5770 - val_loss: 0.6631 - val_accuracy: 0.6294
Epoch 58/1000
331/331 [==============================] - 0s 93us/step - loss: 0.6715 - accuracy: 0.5921 - val_loss: 0.6623 - val_accuracy: 0.6294
Epoch 59/1000
331/331 [==============================] - 0s 98us/step - loss: 0.6673 - accuracy: 0.6042 - val_loss: 0.6654 - val_accuracy: 0.6713
Epoch 60/1000
331/331 [==============================] - 0s 87us/step - loss: 0.6664 - accuracy: 0.6012 - val_loss: 0.6675 - val_accuracy: 0.6294
Epoch 61/1000
331/331 [==============================] - 0s 82us/step - loss: 0.6631 - accuracy: 0.5982 - val_loss: 0.6649 - val_accuracy: 0.6294
Epoch 62/1000
331/331 [==============================] - 0s 74us/step - loss: 0.6649 - accuracy: 0.6042 - val_loss: 0.6636 - val_accurac

331/331 [==============================] - 0s 118us/step - loss: 0.6361 - accuracy: 0.6647 - val_loss: 0.6574 - val_accuracy: 0.5804
Epoch 112/1000
331/331 [==============================] - 0s 81us/step - loss: 0.6364 - accuracy: 0.6647 - val_loss: 0.6382 - val_accuracy: 0.6014
Epoch 113/1000
331/331 [==============================] - 0s 70us/step - loss: 0.6310 - accuracy: 0.6586 - val_loss: 0.6325 - val_accuracy: 0.6643
Epoch 114/1000
331/331 [==============================] - 0s 76us/step - loss: 0.6354 - accuracy: 0.6828 - val_loss: 0.6599 - val_accuracy: 0.5804
Epoch 115/1000
331/331 [==============================] - 0s 71us/step - loss: 0.6328 - accuracy: 0.6616 - val_loss: 0.6323 - val_accuracy: 0.6643
Epoch 116/1000
331/331 [==============================] - 0s 76us/step - loss: 0.6277 - accuracy: 0.6707 - val_loss: 0.6772 - val_accuracy: 0.6643
Epoch 117/1000
331/331 [==============================] - 0s 76us/step - loss: 0.6375 - accuracy: 0.6858 - val_loss: 0.6706 - val_ac

Epoch 167/1000
331/331 [==============================] - 0s 82us/step - loss: 0.5943 - accuracy: 0.7069 - val_loss: 0.5958 - val_accuracy: 0.7133
Epoch 168/1000
331/331 [==============================] - 0s 84us/step - loss: 0.6054 - accuracy: 0.7009 - val_loss: 0.6180 - val_accuracy: 0.7273
Epoch 169/1000
331/331 [==============================] - 0s 82us/step - loss: 0.5935 - accuracy: 0.7221 - val_loss: 0.5840 - val_accuracy: 0.7413
Epoch 170/1000
331/331 [==============================] - 0s 79us/step - loss: 0.5878 - accuracy: 0.7100 - val_loss: 0.5830 - val_accuracy: 0.7343
Epoch 171/1000
331/331 [==============================] - 0s 81us/step - loss: 0.5956 - accuracy: 0.7100 - val_loss: 0.5908 - val_accuracy: 0.7133
Epoch 172/1000
331/331 [==============================] - 0s 82us/step - loss: 0.6019 - accuracy: 0.6918 - val_loss: 0.6109 - val_accuracy: 0.7133
Epoch 173/1000
331/331 [==============================] - 0s 79us/step - loss: 0.5955 - accuracy: 0.6979 - val_loss: 0

Epoch 223/1000
331/331 [==============================] - 0s 74us/step - loss: 0.5456 - accuracy: 0.7402 - val_loss: 0.5624 - val_accuracy: 0.7413
Epoch 224/1000
331/331 [==============================] - 0s 87us/step - loss: 0.5696 - accuracy: 0.7432 - val_loss: 0.6682 - val_accuracy: 0.7343
Epoch 225/1000
331/331 [==============================] - 0s 83us/step - loss: 0.5747 - accuracy: 0.7130 - val_loss: 0.5768 - val_accuracy: 0.7133
Epoch 226/1000
331/331 [==============================] - 0s 75us/step - loss: 0.5649 - accuracy: 0.7402 - val_loss: 0.5870 - val_accuracy: 0.7413
Epoch 227/1000
331/331 [==============================] - 0s 83us/step - loss: 0.5452 - accuracy: 0.7190 - val_loss: 1.2665 - val_accuracy: 0.7203
Epoch 228/1000
331/331 [==============================] - 0s 85us/step - loss: 0.7312 - accuracy: 0.6647 - val_loss: 0.5487 - val_accuracy: 0.7622
Epoch 229/1000
331/331 [==============================] - 0s 76us/step - loss: 0.5658 - accuracy: 0.7069 - val_loss: 0

...

Epoch 726/1000
331/331 [==============================] - 0s 91us/step - loss: 0.4159 - accuracy: 0.7704 - val_loss: 0.4078 - val_accuracy: 0.7692
Epoch 727/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4171 - accuracy: 0.7674 - val_loss: 0.3718 - val_accuracy: 0.7972
Epoch 728/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4402 - accuracy: 0.7704 - val_loss: 0.4212 - val_accuracy: 0.7273
Epoch 729/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4178 - accuracy: 0.7644 - val_loss: 0.4040 - val_accuracy: 0.7762
Epoch 730/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4001 - accuracy: 0.7734 - val_loss: 0.3738 - val_accuracy: 0.8112
Epoch 731/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4056 - accuracy: 0.7825 - val_loss: 0.4356 - val_accuracy: 0.7273
Epoch 732/1000
331/331 [==============================] - 0s 84us/step - loss: 0.4216 - accuracy: 0.7492 - val_loss: 0

Epoch 782/1000
331/331 [==============================] - 0s 91us/step - loss: 0.4058 - accuracy: 0.7704 - val_loss: 0.3866 - val_accuracy: 0.7762
Epoch 783/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3860 - accuracy: 0.7825 - val_loss: 0.3773 - val_accuracy: 0.7832
Epoch 784/1000
331/331 [==============================] - 0s 90us/step - loss: 0.4206 - accuracy: 0.7825 - val_loss: 0.3959 - val_accuracy: 0.7762
Epoch 785/1000
331/331 [==============================] - 0s 79us/step - loss: 0.4081 - accuracy: 0.7674 - val_loss: 0.3806 - val_accuracy: 0.8112
Epoch 786/1000
331/331 [==============================] - 0s 99us/step - loss: 0.3855 - accuracy: 0.7764 - val_loss: 0.3924 - val_accuracy: 0.7762
Epoch 787/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3816 - accuracy: 0.7915 - val_loss: 0.4759 - val_accuracy: 0.7273
Epoch 788/1000
331/331 [==============================] - 0s 75us/step - loss: 0.4250 - accuracy: 0.7553 - val_loss: 0

331/331 [==============================] - 0s 109us/step - loss: 0.4269 - accuracy: 0.7674 - val_loss: 0.4103 - val_accuracy: 0.7692
Epoch 838/1000
331/331 [==============================] - 0s 99us/step - loss: 0.4151 - accuracy: 0.7674 - val_loss: 0.4082 - val_accuracy: 0.7832
Epoch 839/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4095 - accuracy: 0.7764 - val_loss: 0.4013 - val_accuracy: 0.7902
Epoch 840/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4040 - accuracy: 0.7704 - val_loss: 0.3862 - val_accuracy: 0.7762
Epoch 841/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3839 - accuracy: 0.7795 - val_loss: 0.3644 - val_accuracy: 0.8112
Epoch 842/1000
331/331 [==============================] - 0s 82us/step - loss: 0.4016 - accuracy: 0.7764 - val_loss: 0.3668 - val_accuracy: 0.7972
Epoch 843/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3901 - accuracy: 0.7855 - val_loss: 0.3922 - val_ac

Epoch 893/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3817 - accuracy: 0.7825 - val_loss: 0.3503 - val_accuracy: 0.8252
Epoch 894/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3752 - accuracy: 0.7946 - val_loss: 0.3822 - val_accuracy: 0.7832
Epoch 895/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3698 - accuracy: 0.7885 - val_loss: 0.7221 - val_accuracy: 0.6154
Epoch 896/1000
331/331 [==============================] - 0s 83us/step - loss: 0.5286 - accuracy: 0.6767 - val_loss: 0.4698 - val_accuracy: 0.7273
Epoch 897/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4540 - accuracy: 0.7644 - val_loss: 0.4277 - val_accuracy: 0.7902
Epoch 898/1000
331/331 [==============================] - ETA: 0s - loss: 0.3601 - accuracy: 0.90 - 0s 78us/step - loss: 0.4277 - accuracy: 0.7764 - val_loss: 0.4058 - val_accuracy: 0.7902
Epoch 899/1000
331/331 [==============================] - 0s 84us/step - los

331/331 [==============================] - 0s 88us/step - loss: 0.4038 - accuracy: 0.7734 - val_loss: 0.3597 - val_accuracy: 0.8252
Epoch 949/1000
331/331 [==============================] - 0s 95us/step - loss: 0.4166 - accuracy: 0.7825 - val_loss: 0.3448 - val_accuracy: 0.7902
Epoch 950/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3601 - accuracy: 0.7976 - val_loss: 0.3697 - val_accuracy: 0.7832
Epoch 951/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4078 - accuracy: 0.7613 - val_loss: 0.3671 - val_accuracy: 0.7972
Epoch 952/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3810 - accuracy: 0.8006 - val_loss: 0.4641 - val_accuracy: 0.7692
Epoch 953/1000
331/331 [==============================] - 0s 80us/step - loss: 0.5034 - accuracy: 0.7130 - val_loss: 0.4044 - val_accuracy: 0.7692
Epoch 954/1000
331/331 [==============================] - 0s 74us/step - loss: 0.4114 - accuracy: 0.7704 - val_loss: 0.3945 - val_acc

In [106]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 61us/step
combination test accuracy: 77.62%


In [115]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [116]:
model2_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [117]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 388us/step - loss: 7.7969 - accuracy: 0.4924 - val_loss: 0.6918 - val_accuracy: 0.6084
Epoch 2/1000
331/331 [==============================] - 0s 84us/step - loss: 0.7703 - accuracy: 0.5921 - val_loss: 0.7118 - val_accuracy: 0.5734
Epoch 3/1000
331/331 [==============================] - 0s 100us/step - loss: 0.6773 - accuracy: 0.6254 - val_loss: 0.6916 - val_accuracy: 0.6224
Epoch 4/1000
331/331 [==============================] - 0s 97us/step - loss: 0.6827 - accuracy: 0.5831 - val_loss: 0.6792 - val_accuracy: 0.6294
Epoch 5/1000
331/331 [==============================] - 0s 97us/step - loss: 0.6713 - accuracy: 0.6042 - val_loss: 0.6707 - val_accuracy: 0.6084
Epoch 6/1000
331/331 [==============================] - 0s 101us/step - loss: 0.6663 - accuracy: 0.6526 - val_loss: 0.6690 - val_accuracy: 0.6643
Epoch 7/1000
331/331 [==============================] - 0s 94us/step - loss: 0.66

331/331 [==============================] - 0s 120us/step - loss: 0.5946 - accuracy: 0.6949 - val_loss: 0.6060 - val_accuracy: 0.6783
Epoch 57/1000
331/331 [==============================] - 0s 111us/step - loss: 0.5879 - accuracy: 0.6949 - val_loss: 0.6371 - val_accuracy: 0.6573
Epoch 58/1000
331/331 [==============================] - 0s 99us/step - loss: 0.5963 - accuracy: 0.6737 - val_loss: 0.6736 - val_accuracy: 0.6643
Epoch 59/1000
331/331 [==============================] - 0s 78us/step - loss: 0.5926 - accuracy: 0.7069 - val_loss: 0.5812 - val_accuracy: 0.6853
Epoch 60/1000
331/331 [==============================] - 0s 87us/step - loss: 0.5917 - accuracy: 0.6918 - val_loss: 0.5735 - val_accuracy: 0.6713
Epoch 61/1000
331/331 [==============================] - 0s 88us/step - loss: 0.5748 - accuracy: 0.7039 - val_loss: 0.6210 - val_accuracy: 0.6573
Epoch 62/1000
331/331 [==============================] - 0s 88us/step - loss: 0.5897 - accuracy: 0.7009 - val_loss: 0.5728 - val_accurac

331/331 [==============================] - 0s 97us/step - loss: 0.4977 - accuracy: 0.7523 - val_loss: 0.4982 - val_accuracy: 0.7622
Epoch 113/1000
331/331 [==============================] - 0s 87us/step - loss: 0.5031 - accuracy: 0.7492 - val_loss: 0.4984 - val_accuracy: 0.7413
Epoch 114/1000
331/331 [==============================] - 0s 101us/step - loss: 0.5346 - accuracy: 0.7221 - val_loss: 0.7814 - val_accuracy: 0.6643
Epoch 115/1000
331/331 [==============================] - 0s 87us/step - loss: 0.5459 - accuracy: 0.7190 - val_loss: 0.4955 - val_accuracy: 0.7413
Epoch 116/1000
331/331 [==============================] - 0s 78us/step - loss: 0.5244 - accuracy: 0.7221 - val_loss: 0.7327 - val_accuracy: 0.6643
Epoch 117/1000
331/331 [==============================] - 0s 80us/step - loss: 0.5331 - accuracy: 0.7341 - val_loss: 0.4924 - val_accuracy: 0.7413
Epoch 118/1000
331/331 [==============================] - 0s 84us/step - loss: 0.5134 - accuracy: 0.7462 - val_loss: 0.5616 - val_ac

Epoch 168/1000
331/331 [==============================] - 0s 92us/step - loss: 0.4776 - accuracy: 0.7492 - val_loss: 0.4705 - val_accuracy: 0.7483
Epoch 169/1000
331/331 [==============================] - 0s 88us/step - loss: 0.4800 - accuracy: 0.7644 - val_loss: 0.5035 - val_accuracy: 0.7273
Epoch 170/1000
331/331 [==============================] - 0s 85us/step - loss: 0.4714 - accuracy: 0.7795 - val_loss: 0.5122 - val_accuracy: 0.7483
Epoch 171/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4693 - accuracy: 0.7704 - val_loss: 0.4681 - val_accuracy: 0.7692
Epoch 172/1000
331/331 [==============================] - 0s 124us/step - loss: 0.4887 - accuracy: 0.7523 - val_loss: 0.4606 - val_accuracy: 0.7483
Epoch 173/1000
331/331 [==============================] - 0s 75us/step - loss: 0.4727 - accuracy: 0.7644 - val_loss: 0.4432 - val_accuracy: 0.7762
Epoch 174/1000
331/331 [==============================] - 0s 73us/step - loss: 0.4885 - accuracy: 0.7341 - val_loss: 

Epoch 224/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4467 - accuracy: 0.7644 - val_loss: 0.4185 - val_accuracy: 0.7692
Epoch 225/1000
331/331 [==============================] - 0s 90us/step - loss: 0.4364 - accuracy: 0.7644 - val_loss: 0.4047 - val_accuracy: 0.8252
Epoch 226/1000
331/331 [==============================] - 0s 89us/step - loss: 0.5151 - accuracy: 0.7583 - val_loss: 0.4909 - val_accuracy: 0.7483
Epoch 227/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4515 - accuracy: 0.7764 - val_loss: 0.4076 - val_accuracy: 0.7692
Epoch 228/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4351 - accuracy: 0.7644 - val_loss: 0.4225 - val_accuracy: 0.8042
Epoch 229/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4322 - accuracy: 0.7795 - val_loss: 0.4833 - val_accuracy: 0.7273
Epoch 230/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4314 - accuracy: 0.7704 - val_loss: 0

...

Epoch 724/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3626 - accuracy: 0.7915 - val_loss: 0.3534 - val_accuracy: 0.7762
Epoch 725/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3527 - accuracy: 0.7915 - val_loss: 0.3765 - val_accuracy: 0.7832
Epoch 726/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3665 - accuracy: 0.7915 - val_loss: 0.3560 - val_accuracy: 0.7762
Epoch 727/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3525 - accuracy: 0.7915 - val_loss: 0.3547 - val_accuracy: 0.7762
Epoch 728/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3603 - accuracy: 0.7946 - val_loss: 0.3485 - val_accuracy: 0.7832
Epoch 729/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3535 - accuracy: 0.7885 - val_loss: 0.3681 - val_accuracy: 0.7972
Epoch 730/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3676 - accuracy: 0.7915 - val_loss: 0

Epoch 780/1000
331/331 [==============================] - 0s 103us/step - loss: 0.3697 - accuracy: 0.7855 - val_loss: 0.3587 - val_accuracy: 0.7832
Epoch 781/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3504 - accuracy: 0.7915 - val_loss: 0.3567 - val_accuracy: 0.8042
Epoch 782/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3479 - accuracy: 0.8066 - val_loss: 0.3757 - val_accuracy: 0.7762
Epoch 783/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3694 - accuracy: 0.7825 - val_loss: 0.4117 - val_accuracy: 0.7273
Epoch 784/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3609 - accuracy: 0.7885 - val_loss: 0.3638 - val_accuracy: 0.7483
Epoch 785/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3630 - accuracy: 0.7946 - val_loss: 0.3584 - val_accuracy: 0.7902
Epoch 786/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3505 - accuracy: 0.7855 - val_loss: 

Epoch 836/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3574 - accuracy: 0.7915 - val_loss: 0.3690 - val_accuracy: 0.7972
Epoch 837/1000
331/331 [==============================] - 0s 101us/step - loss: 0.3707 - accuracy: 0.7915 - val_loss: 0.3575 - val_accuracy: 0.7972
Epoch 838/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3554 - accuracy: 0.8066 - val_loss: 0.5131 - val_accuracy: 0.7273
Epoch 839/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3652 - accuracy: 0.8006 - val_loss: 0.4711 - val_accuracy: 0.7343
Epoch 840/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3663 - accuracy: 0.7885 - val_loss: 0.3675 - val_accuracy: 0.7972
Epoch 841/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3538 - accuracy: 0.7976 - val_loss: 0.3405 - val_accuracy: 0.8042
Epoch 842/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3646 - accuracy: 0.7764 - val_loss: 

Epoch 892/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3574 - accuracy: 0.8006 - val_loss: 0.3660 - val_accuracy: 0.8042
Epoch 893/1000
331/331 [==============================] - 0s 92us/step - loss: 0.3647 - accuracy: 0.7915 - val_loss: 0.3860 - val_accuracy: 0.7552
Epoch 894/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3447 - accuracy: 0.7946 - val_loss: 0.3635 - val_accuracy: 0.7832
Epoch 895/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3567 - accuracy: 0.8036 - val_loss: 0.3501 - val_accuracy: 0.7762
Epoch 896/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3351 - accuracy: 0.8066 - val_loss: 0.5921 - val_accuracy: 0.7203
Epoch 897/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3609 - accuracy: 0.7976 - val_loss: 0.3492 - val_accuracy: 0.7902
Epoch 898/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3594 - accuracy: 0.7855 - val_loss: 0

Epoch 948/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3649 - accuracy: 0.7946 - val_loss: 0.3501 - val_accuracy: 0.7902
Epoch 949/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3439 - accuracy: 0.7946 - val_loss: 0.5076 - val_accuracy: 0.7832
Epoch 950/1000
331/331 [==============================] - 0s 94us/step - loss: 0.3883 - accuracy: 0.8066 - val_loss: 0.3896 - val_accuracy: 0.7762
Epoch 951/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3505 - accuracy: 0.7885 - val_loss: 0.3441 - val_accuracy: 0.8252
Epoch 952/1000
331/331 [==============================] - 0s 97us/step - loss: 0.3421 - accuracy: 0.8036 - val_loss: 0.3534 - val_accuracy: 0.7902
Epoch 953/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3499 - accuracy: 0.7855 - val_loss: 0.3607 - val_accuracy: 0.7762
Epoch 954/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3473 - accuracy: 0.7946 - val_loss: 0

In [118]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

143/143 [==============================] - 0s 129us/step
combination test accuracy: 80.42%


In [119]:
############ Logistic Regression ############

In [120]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [121]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [122]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 96.43%


In [123]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [124]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 94.41%


In [125]:
############## Random Forest ##############

In [126]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [127]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 98.21%


In [128]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [129]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 97.20%


In [130]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [131]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.98113208 1.         0.98076923 0.98039216 0.98039216]
0.9845371239932839


In [132]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98507463 0.96969697 0.98484848 1.         0.93939394]
0.9758028041610132
